In [1]:
### Library functions
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import matplotlib.pyplot as plt
import random 
from matplotlib import animation
import networkx.algorithms.centrality as nx_centrality
from collections import deque
from matplotlib import animation
import math

/Users/fengboyang/opt/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

## Training dataset

The global parameters of the generate_random_graph function:
1. FAKE_DIFF_ITER: the fake nodes diffusion iteration
2. FAKE_SEED_NUM: the initial fake seed set number
3. NODE_NUM: the graph nodes number
4. EDGE_NUM: the graph edges number 
5. M_INDEX: the generate graph method index

In [2]:
def generate_random_graph(difusion_iteration, method_index,fake_seed_num, num_nodes , num_edges , probability = 0.5,m = 2, radius = None, k_nearest_neighbor = None, degree = None, seed = None):
    random_graph = nx.Graph()
    if method_index == 0:
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)

    elif method_index == 1:
        random_graph = nx.erdos_renyi_graph(num_nodes, probability)

    elif method_index == 2:
         # m means the edges number from the create point to existing point
        random_graph = nx.barabasi_albert_graph(num_nodes, m)

    elif method_index == 3:
        random_graph = nx.watts_strogatz_graph(num_nodes, k_nearest_neighbor, probability)
#     elif method_index == 4:
#         random_graph=nx.gnc_graph(num_nodes)

#     elif method_index == 4:
#         random_graph = nx.random_geometric_graph(num_nodes, radius)   
        
#     elif method_index == 5:
#         random_graph = nx.random_regular_graph(degree, num_nodes)

#     elif method_index == 7:
#         #p1 = nodes in the tree connected,  p2 = nodes in the tree body connected
#         random_graph = nx.random_lobster(num_nodes, probability, probability)

#     elif method_index == 8:
#         p = probability # probability connected with its nearest neighbor
#         q = probability # probability connected with other nodes
#         random_graph = nx.newman_watts_strogatz_graph(num_nodes, p, q)

#     elif method_index == 9:
#         random_graph = nx.create_powerlaw_cluster_graph(num_nodes, k_nearest_neighbor, probability, seed=seed)

#     elif method_index == 10:
#         # A random maximal planar graph is a graph in which every possible edge that can be added without creating a cycle is included, 
#         # resulting in a planar graph with the maximum number of edges. 
#         random_graph = nx.random_maximal_planar_graph(num_nodes)

#     elif method_index == 11:
#         random_graph = nx.random_threshold_graph(num_nodes, probability)

#     elif method_index == 12:
#         random_graph = nx.create_forest_fire_graph(num_nodes, probability, probability, seed=seed)

#     elif method_index == 13:
#         k = k_nearest_neighbor # This k is different with KNN, it generates a scale-free network where new nodes are added with k initial edges
#         random_graph = nx.extended_barabasi_albert_graph(num_nodes, k, probability)


#     elif method_index == 15:
#         # lollipop graph
#         m = num_nodes // 2  # Number of nodes in the complete graph
#         n = num_nodes - m - 1  # Number of nodes in the cycle graph

#         G_complete = nx.complete_graph(m)
#         G_cycle = nx.cycle_graph(n)

#         # Connect the complete graph and cycle graph
#         G_complete.add_edge(m-1, m)
#         random_graph = nx.compose(G_complete, G_cycle)
    
    
    for (u,v) in random_graph.edges:
        if FIXED == True:
            random_graph.edges[u, v]["weight"] = PROBABILITY
        else:
            random_graph.edges[u, v]["weight"] =  random.uniform(0,1)
    for node in random_graph.nodes():
        random_graph.nodes[node]['state'] = 0 # 0初始状态；1:true；2:fake
    node_number = random_graph.number_of_nodes()
    start_node_index = [0] *fake_seed_num
    index_range = range(0,  node_number)
    fake_seed_set = random.sample(index_range, fake_seed_num)
    fake_active_nodes = fake_seed_set.copy()
    fake_nodes_set = fake_seed_set.copy()
    for _ in range(difusion_iteration):
        temp_fake = []
        for v in fake_active_nodes:
            for nbr in random_graph.neighbors(v): 
                if random_graph.nodes[nbr]['state'] == 0 : 
                    edge_data = random_graph.get_edge_data(v, nbr)
                    if random.uniform(0, 1) < edge_data['weight']:
                        fake_nodes_set.append(nbr)
                        temp_fake.append(nbr)
                        random_graph.nodes[nbr]['state'] = 2
        fake_active_nodes = temp_fake.copy()
        temp_fake.clear()    
    for node in fake_nodes_set:
        random_graph.nodes[node]['state'] = 2
        
    return random_graph,fake_active_nodes,fake_seed_set,fake_nodes_set


## Create and normalize test data

In [3]:
# !pip install scanpy

In [4]:
import scanpy as sc 
import scipy as sp
import scipy.sparse  # call as sp.sparse

# filename_ca_HepTh = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\ca-HepTh.mtx"
# filename_ca_GrQc = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\ca-GrQc.mtx"
# filename_tech_p2p_gnutella = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\tech-p2p-gnutella.mtx"
filename_ca_HepTh = 'ca-HepTh.mtx'
filename_ca_GrQc = 'ca-GrQc.mtx'
filename_tech_p2p_gnutella = 'tech-p2p-gnutella.mtx'

In [5]:
def read_graph(filename):
    adata = sc.read(filename)
    G = nx.from_scipy_sparse_array(adata.X, create_using=nx.MultiGraph)
    print("node number:",G.number_of_nodes())
    print("edge number:",G.number_of_edges())
    return G, G.number_of_nodes(), G.number_of_edges()

In [6]:
G_ca_HepTh,N_ca_HepTh,E_ca_HepTh= read_graph(filename_ca_HepTh)

node number: 9877
edge number: 25998


In [7]:
G_ca_GrQc,N_ca_GrQc,E_ca_GrQc = read_graph(filename_ca_GrQc)

node number: 5242
edge number: 14496


In [8]:
G_tech_p2p_gnutella,N_tech_p2p_gnutella,E_tech_p2p_gnutella = read_graph(filename_tech_p2p_gnutella)

node number: 62561
edge number: 147878


In [9]:
def normalization_graph(graph):
    graph_node_number = graph.number_of_nodes()
    graph_edge_number = graph.number_of_edges()
    for node in graph.nodes():
        graph.nodes[node]['state'] = 0
    for (u,v) in graph.edges:
        graph.edges[u,v]['weight'] = random.uniform(0,1)
    return graph,graph_node_number,graph_edge_number

In [10]:
def fake_nodes_diffusion(graph,fake_seed_num,iteration):
    node_number = graph.number_of_nodes()
    start_node_index = [0] *fake_seed_num
    index_range = range(0,  node_number)
    fake_seed_set = random.sample(index_range, fake_seed_num)
    fake_active_nodes = fake_seed_set.copy()
    fake_nodes_set = fake_seed_set.copy()
    for node in fake_seed_set:
        graph.nodes[node]['state'] = 2
    #fake news diffusion
    for _ in range(iteration):
        temp_fake = []
        for v in fake_active_nodes:
            for nbr in graph.neighbors(v): 
                if graph.nodes[nbr]['state'] == 0 : 
                    edge_data = graph.get_edge_data(v, nbr)
                    if random.uniform(0, 1) < edge_data['weight']:
                        fake_nodes_set.append(nbr)
                        temp_fake.append(nbr)
                        graph.nodes[nbr]['state'] = 2
        fake_active_nodes = temp_fake.copy()  
    for node in fake_nodes_set:
        random_graph.nodes[node]['state'] = 2
    return graph,fake_active_nodes

In [11]:
def statistic_draw_graph(Graph,draw = False):
    color_list = []
    fake = 0
    true = 0
    normal = 0
    for node in Graph.nodes():
        if Graph.nodes[node]['state'] == 0:
            normal += 1
            color_list.append('blue')
        elif Graph.nodes[node]['state'] == 1:
            true += 1
            color_list.append('green')
        elif Graph.nodes[node]['state'] == 2:
            fake += 1
            color_list.append('red')
    print("normal nodes number:",normal)
    print("true nodes number:",true)
    print("fake nodes number:",fake)
    print("the edge of graph:",Graph.number_of_edges())
    #nx.draw(Graph, node_color= color_list) 
    if draw:
        nx.draw_circular(Graph, node_color= color_list)
        plt.show()

# Environment Class

The global parameters in the Environment Class:
1. ITERATION: control the diffusion times during the true and fake nodes antagonistic process
2. MAX_STEP: control the max step of this epoch
3. SEED_SIZE: the initial size of the seed set, the initial parameters of Env class.

The input variable of the Env() class:
1. graph: the training graph or testing graph
2. seed_size: SEED_SIZE;the initial size of the seed set
3. fake_set: the initial fake

The functions in the Env class:
1. reset: reset the parameters
2. step: the experiment excuation progress
3. select_initial_seeds: select the initial seed randomly
4. select_initial_seeds_rules: select the intial seed set according to different criterion
5. add_seed: add the seed nodes according to the action number from the agent
6. get_state: get the current state of the graph
7. diffusion_process: the procees of the true and fake nodes antagonistic
8. get_image: draw the image of the graph

In [90]:
class Env():
    def __init__(self, graph,seed_size,fake_set):
        self.graph_initial = graph
        self.seed_size = seed_size
        self.seed_set = []
        self.fake_set = fake_set
        self.state = None
        self.reset()

    def reset(self):
        #select the seed set:
        self.graph = self.graph_initial.copy()
        self.step_count = 0
        self.add_number = 0
        self.seed_set, action = self.select_initial_seeds_rules(self.graph,self.seed_size)
        self.true_active_nodes = self.seed_set.copy()
        self.fake_active_nodes = self.fake_set.copy()
        self.state = self.get_state()
        return self.state, action
    
    def step(self,action):
        count_state0 = 0
        for node in self.graph.nodes():
            if self.graph.nodes[node]['state'] == 0:
                count_state0 += 1
        if action > 0 and (len(self.seed_set) + ADD_SEED_NUMBER <= self.graph.number_of_nodes()) and count_state0 >= ADD_SEED_NUMBER:
            self.add_number += 1
            centrality_methods = ["None", "random", "degree", "closeness", "betweenness", "eigenvector"]
            centrality_method = centrality_methods[action]
            self.seed_set = self.add_seed(self.graph,self.seed_set,action,ADD_SEED_NUMBER)
            if(self.true_active_nodes[-ADD_SEED_NUMBER:]!= self.seed_set[-ADD_SEED_NUMBER:]):
                self.true_active_nodes.extend(self.seed_set[-ADD_SEED_NUMBER:])
        self.graph,self.true_active_nodes,self.fake_active_nodes = self.diffusion_process(self.graph,
                                                                                     self.true_active_nodes,
                                                                               self.fake_active_nodes,ITERATION)
        #print("current image:")
        #self.get_image(self.graph)
        
        # terminal condition                                                                    
        all_true_nodes = []
        all_fake_nodes = []
        normal_nodes= []                                                                             
        for node in self.graph.nodes():
            if self.graph.nodes[node]['state'] == 1:
                all_true_nodes.append(node)
            elif self.graph.nodes[node]['state'] == 2:
                all_fake_nodes.append(node)
            elif self.graph.nodes[node]['state'] == 0:
                normal_nodes.append(node)
        number_nodes = len(self.graph.nodes())
        number_true_nodes = len(all_true_nodes) 
        number_fake_nodes = len(all_fake_nodes)
        # terminal condition: over the MAX_STEP / less 1% nodes are normal nodes
        terminated = bool(self.step_count >= MAX_STEP 
                        or len(normal_nodes)<=0.01 * number_nodes) 
#         penatly = math.pow(1.2,self.add_number)
#         print("penatly:",penatly)
#         print(len(all_true_nodes)  - len(all_fake_nodes))
        penatly = math.pow(1.05,self.add_number)
        if not terminated:
            if len(all_true_nodes)>len(all_fake_nodes):
                reward = (len(all_true_nodes)  - len(all_fake_nodes) - penatly + 5)/(len(all_true_nodes) + len(all_fake_nodes))
                #print(">:",reward)
            else:
                reward = (len(all_true_nodes)  - len(all_fake_nodes) - penatly)/(len(all_true_nodes) + len(all_fake_nodes))
                
                #print("<:",reward)
            done = False
            self.step_count +=1
        else: 
            # when this iteration is end, give a little big reward.
            reward = 30
            done = True
#         if not terminated:
#             if len(all_true_nodes)>len(all_fake_nodes):
#                 reward = (len(all_true_nodes)  - len(all_fake_nodes) - penatly )/(len(all_true_nodes) + len(all_fake_nodes))#+0.7
#                 print(">:",reward)
#             else:
#                 reward = (len(all_true_nodes)  - len(all_fake_nodes))/(len(all_true_nodes) + len(all_fake_nodes)) #-0.5
#                 print("<:",reward)
#             done = False
#             self.step_count +=1
#         else:
#             reward = 10 +  (MAX_STEP - self.step_count) * 0.25
#             done = True
            
#             self.step_count >= MAX_STEP: 
#             # when this iteration is end, give a little big reward.
#             reward = 20
#             done = True
#         elif len(normal_nodes)<=0.01 * number_nodes:
#             reward = 30
#             done = True
        self.state = self.get_state()
        return self.state, reward ,done 
    # select the initial seed set randomly
    def select_initial_seeds(self, graph, seed_number):
        node_number = graph.number_of_nodes()
        start_node_index = [0] *seed_number
        index_range = range(0,  node_number)
        start_node_index = random.sample(index_range, seed_number)
        for node in graph.nodes():
            if node in start_node_index:
                graph.nodes[node]['state'] = 1
        return start_node_index
    # select the initial seed set according to different rules
    def select_initial_seeds_rules(self,graph,seed_number):
        # 1 random 2 "degree",3 "closeness", 4"betweenness", 5"eigenvector"
        action = random.randint(0,5)
        start_node_index = []
        centrality_scores = []
        if action == 0:   
            node_number = graph.number_of_nodes()
            start_node_index = [0] *seed_number
            index_range = range(0,  node_number)
            start_node_index = random.sample(index_range, seed_number)
        else:
            if action == 1:
                centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
            elif action == 2:
                centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
            elif action == 3:
                centrality_scores = list(nx_centrality.closeness_centrality(self.graph).values())
            elif action == 4:
                centrality_scores = list(nx_centrality.betweenness_centrality(self.graph).values())
            elif action == 5:
                centrality_scores = list(nx_centrality.eigenvector_centrality(self.graph).values())
            for _ in range(seed_number):
                max_number = max(centrality_scores)
                index = centrality_scores.index(max_number)
                while(graph.nodes[index]['state'] != 0):
                    centrality_scores[index] = -1
                    max_number = max(centrality_scores)
                    index = centrality_scores.index(max_number)  
                start_node_index.append(index)
                centrality_scores[index] = -1
        for node in graph.nodes():
            if node in start_node_index:
                graph.nodes[node]['state'] = 1
        return start_node_index,action
        
    def add_seed(self,graph,seed_set,action,add_number):
        # 1 random 2 "degree",3 "closeness", 4"betweenness", 5"eigenvector"
        new_seed_set = []
        new_seed_set = seed_set.copy()
        centrality_scores = []
        state0 = 0
        for node in graph.nodes():
            if graph.nodes[node]['state'] == 0:
                state0 += 1
        if state0 >= ADD_SEED_NUMBER:
            if action == 1:
                node_number = graph.number_of_nodes()
                for _ in range(ADD_SEED_NUMBER):
                    index = random.randint(0,node_number-1)
                    while (index in seed_set) or (graph.nodes[index]['state'] in [1,2])  :
                        index = random.randint(0,node_number-1)
                    new_seed_set.append(index)
                    graph.nodes[index]['state']  = 1

            else: 
                if action == 2:
                    centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
                elif action == 3:
                    centrality_scores = list(nx_centrality.closeness_centrality(self.graph).values())
                elif action == 4:
                    centrality_scores = list(nx_centrality.betweenness_centrality(self.graph).values())
                elif action == 5:
                    centrality_scores = list(nx_centrality.eigenvector_centrality(self.graph).values())
                for node in seed_set:
                    centrality_scores[node] = -1
                for _ in range(ADD_SEED_NUMBER):
                    index = centrality_scores.index(max(centrality_scores))
                    while(graph.nodes[index]['state'] != 0):
                        index = centrality_scores.index(max(centrality_scores))
                        # print(index)
                        centrality_scores[index] = -1
                    # print("add_seed, index:",index)
                    new_seed_set.append(index)
                    graph.nodes[index]['state'] = 1
                    centrality_scores[index] = -1
        return new_seed_set
    def get_state(self):
        node_state = []
        for node in self.graph.nodes():
            node_state.append(self.graph.nodes[node]['state'])
        return nx.to_numpy_array(self.graph), len(self.seed_set), node_state
    
    def diffusion_process(self, G ,true_active_set,fake_active_set,itertaion):
        G = self.graph.copy()
        true_active_nodes = true_active_set.copy()
        fake_active_nodes = fake_active_set.copy()
        # print("true_active_nodes begin:",true_active_nodes)
        # print("fake_active_nodes begin:",fake_active_nodes)
        for _ in range(itertaion):
            tmp_true_nodes ={}
            for v in true_active_nodes: 
                for nbr in G.neighbors(v):
                    if G.nodes[nbr]['state'] in [0]:
                        edge_data = G.get_edge_data(v, nbr)
                        random_possibility = random.uniform(0, 1)
                        if random_possibility < edge_data['weight']:
                            G.nodes[nbr]['state'] == 3
                            tmp_true_nodes.update({nbr:random_possibility})
                            true_active_set.append(nbr)
                    else:
                        continue
            tmp_fake_nodes ={}
            for v in fake_active_nodes:
                for nbr in G.neighbors(v):
                    if G.nodes[nbr]['state'] in [0,3]:
                        edge_data = G.get_edge_data(v, nbr)
                        random_possibility = random.uniform(0, 1)
                        if random_possibility < edge_data['weight']:
                            tmp_fake_nodes.update({nbr:random_possibility})
                            fake_active_set.append(nbr)
                            G.nodes[nbr]['state'] == 4
            true_active_nodes = list(tmp_true_nodes.keys())
            fake_active_nodes = list(tmp_fake_nodes.keys())
            for node in tmp_true_nodes.keys():
                if node in tmp_fake_nodes.keys() :
                    if tmp_true_nodes.get(node) >= tmp_fake_nodes.get(node):
                        fake_active_set.remove(node)
                        fake_active_nodes.remove(node)
                    elif tmp_true_nodes.get(node) < tmp_fake_nodes.get(node):
                        true_active_set.remove(node)
                        true_active_nodes.remove(node)
                else:
                    continue
            for node in true_active_set:
                G.nodes[node]['state'] = 1
            for node in fake_active_set:
                G.nodes[node]['state'] = 2
            # print("true_active_nodes",true_active_set)
            # print("fake_active_nodes",fake_active_set)
        return G,true_active_nodes,fake_active_nodes

        
    def get_image(self,Graph):
        color_list = []
        for node in Graph.nodes():
            if Graph.nodes[node]['state'] == 0:
                color_list.append('blue')
            elif Graph.nodes[node]['state'] == 1:
                color_list.append('green')
            elif Graph.nodes[node]['state'] == 2:
                color_list.append('red')
        #nx.draw(Graph, node_color= color_list) 
        nx.draw_circular(Graph, node_color= color_list)
        plt.show()

# Agent

##  Device: GPU/CPU
Note:

if you use Macbook with Apple M1/M2 core, please run the code in the Mac Device;

if you use Windows with CUDA core, please run the code in the Windows Device;

### Mac Device

In [13]:
# # if use MAC(Apple M1/M2) please run the follow codes
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


### Windows Device

In [15]:
import torch
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: False


In [16]:
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name()
    print("GPU Device Name:", device_name)

In [17]:
if torch.backends.cudnn.is_available():
    print("cuDNN Available")

In [18]:
# if use CUDA please run the follow codes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## DQN Agent

The input variable of the DQNAgent() class link to the global parameters:
1. state_dim: STATE_DIM; Dimension of state
2. action_dim: ACT_DIM; Dimension of action space 
3. learning_rate: LR;
4. gamma: GAMMA
5. epsilon: EPSILON
6. epsilon_decay: EPSILON_DECAY

In [14]:
class DQNAgent:
    def __init__(self, state_dim, action_dim, learning_rate, gamma, epsilon, epsilon_decay):
        self.q_network = QNetwork(state_dim, action_dim).to(device)
        self.target_network = QNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.loss_fn = nn.MSELoss()
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=10000)

    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, action_dim)
        else:
            state = torch.tensor(state, dtype=torch.float32).to(device)
            q_values = self.q_network(state)
#           q_values = self.q_network(torch.tensor(state, dtype=torch.float32))
            return torch.argmax(q_values.cpu()).item()

    def remember(self, state, action, reward, next_state, done):
        #state = torch.tensor(state).float().unsqueeze(0)  # ensure state is a tensor
        self.memory.append((state, action, reward, next_state, done))

    def train(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            #print("next_state:", next_state)
            #state = torch.tensor(state, dtype=torch.float32).to(device)
            state_tensor = state_to_tensor(state).to(device)
            #print(state_tensor.shape)
            #print(self.q_network(state_tensor).shape)
            action_index = torch.tensor(action, dtype=torch.long)
            target = self.q_network(state_tensor)[0, action_index]
            #target = self.q_network(state_tensor)[action]
            if done:
                target_value = reward
            else:
                next_state_tensor = state_to_tensor(next_state).to(device)
                #next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
                next_q_values = self.target_network(next_state_tensor).detach()
                target_value = reward + self.gamma * torch.max(next_q_values).item()

            #loss = self.loss_fn(target, torch.tensor(float(target_value)))
            loss = self.loss_fn(target, torch.tensor(float(target_value)).to(device))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        self.epsilon *= self.epsilon_decay

    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

    def save(self, path):
        torch.save(self.q_network.state_dict(), path)

    def load(self, path):
        self.q_network.load_state_dict(torch.load(path))
        self.target_network.load_state_dict(self.q_network.state_dict())

### Transform array state to tensor state

In [15]:
def state_to_tensor(state):
    graph, num_seed_nodes, node_state = state
    
    graph_tensor = torch.tensor(graph, dtype=torch.float32).view(-1).unsqueeze(0)
    num_seed_nodes_tensor = torch.tensor([num_seed_nodes], dtype=torch.float32).unsqueeze(0)
    node_state_tensor = torch.tensor(node_state, dtype=torch.float32).unsqueeze(0)
    
#     state_tensor = torch.cat((graph_tensor, num_seed_nodes_tensor, node_state_tensor), dim=1)
    state_tensor = torch.cat((num_seed_nodes_tensor, node_state_tensor), dim=1)
    
    return state_tensor

## Neural Network

In [16]:
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)# (2500 + 50 + 2) (10000 + 100 + 2)
        
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training Function
The input valiable of train_dqn function:

1. agent: the initialized Agent class
2. env: the initialized Env class
3. batch_size: BATCH_SIZE
4. update_target_every: UPDATE;How many iterations to update the network
5. dqn_agent_name: the name of DQN agent, the file extension is '.pth'
6. brenchmark_action: Start benchmark comparison and specifying an action(1-5)
7. Demo: Whether state should be displayed or saved each time

In [17]:
def train_dqn(agent, env, episodes, batch_size, update_target_every,dqn_agent_name  = None,brenchmark_action = None,Demo = False):
    state_history= []
    rewards_list = []
    for episode in range(episodes):
        state,init_action = env.reset()
        if brenchmark_action is not None:
            init_action = brenchmark_action
        #print(state)
#         state = torch.tensor(state).float().unsqueeze(0)  # ensure state is a tensor
        total_reward = 0
        done = False

        while not done:
            state_tensor = state_to_tensor(state)
            #print(state_tensor)
            if brenchmark_action is not None:
                action = brenchmark_action
            else:
                action = agent.get_action(state_tensor)
            next_state, reward, done = env.step(action)
            #print(next_state)
            agent.remember(state, action, reward, next_state, done)
            agent.train(batch_size)
            total_reward += reward
            state = next_state
            if Demo:
                state_history.append(state)  # Add current state to state_history
        if not Demo:
            state_history.append(state)
        rewards_list.append(total_reward)
        if episode % update_target_every == 0:
            agent.update_target_network()

        print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")
    if dqn_agent_name is not None:
        agent.save(dqn_agent_name)
    return state_history, rewards_list

# Testing Function

In [18]:
def test_dqn(agent, env, episodes):
    for episode in range(episodes):
        state,init_action = env.reset()
        total_reward = 0
        done = False

        while not done:
            state_tensor = state_to_tensor(state)
            action = agent.get_action(state_tensor)
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state

        print(f"Test Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")

# Display functions

## Display the training or testing process

In [19]:
import matplotlib.pyplot as plt
from matplotlib import animation

In [20]:
def state2graph(state):
    graph = nx.from_numpy_array(state[0])
    #print(graph.nodes())
    for node in graph.nodes():
        graph.nodes[node]['state'] = state[2][node]
    return graph

In [21]:
def create_color_list(graph):
    color_list = []
    #create color list
    #print(graph)
    for node in graph.nodes():
        if graph.nodes[node]['state'] == 0:
            color_list.append('blue')
        elif graph.nodes[node]['state'] == 1:
            color_list.append('green')
        elif graph.nodes[node]['state'] == 2:
            color_list.append('red')
    return color_list

In [22]:
def define_pos(state):
    graph = nx.from_numpy_array(state[0])
    position = spring_layout(graph)
    position = nx.circular_layout(graph)   
    return position

In [23]:
def display(state):
    fig, ax = plt.subplots()
    graph = state2graph(state[0])
    pos = define_pos(state[0])
    color_list = create_color_list(graph)
    nx.draw(graph, pos, node_color= color_list)
    
    def animate(frame):
        ax.clear()
        #print("frame,",frame)
        #print(state[frame])
        graph = state2graph(state[frame])
        color_list = create_color_list(graph)
        nx.draw(graph, pos, node_color= color_list)
    anim = animation.FuncAnimation(fig, animate, frames=len(state), interval=200)
    anim.save('train.gif', writer='PillowWriter')
    anim.save("train.mp4",writer='ffmpeg')
    
    plt.show()

## Show the change of reward

In [36]:
def reward_image(rewards_list):
    x = range(1,ALL_EPISODES+1)
    print(x)
    plt.plot(x,rewards_list,'s-')
    plt.xlabel("Iteration")
    plt.ylabel("Total reward")
    plt.show()

# Experiment

## 1. 比较基准值
fake传播次数为1
初始fake节点数为n/n+1
构图方法为随机

In [144]:
# the Global parameter of generate_random_graph
FAKE_DIFF_ITER = 1 # the fake nodes diffusion iteration
FAKE_SEED_NUM = 5 # the initial fake seed set number
NODE_NUM = 500 # the intial graph nodes number
EDGE_NUM = 1500 # the intial graph edges number
M_INDEX = 0 # the generate graph method index

# the Global parameter of Env class
ITERATION = 1 #control the diffusion times during the true and fake nodes antagonistic process
MAX_STEP = 40 # control the max step of this epoch
SEED_SIZE = 10 # the initial size of the seed set, the initial parameters of Env class.
ADD_SEED_NUMBER = 1 # the number of nodes that are added to seed set

# the Global parameter of Agentclass
state_dim = NODE_NUM + 1 #state_dim: Dimension of state
action_dim = 6 #action_dim:  Dimension of action space
LR = 0.001#learning_rate: 
GAMMA = 0.99 # gamma:
EPSILON = 1.0# epsilon: 
EPSILON_DECAY = 0.995 # epsilon_decay: 
# the Global parameter of train and test function
batch_size = 128 #batch_size: 
UPDATE = 10 #update_target_every: 
ALL_EPISODES = 200 # total number of iteration times

FIXED = False

In [145]:
intial_train_graph_bench,active_fake_set_bench,intial_fake_set_bench,all_fake_nodes_bench = generate_random_graph(FAKE_DIFF_ITER,M_INDEX,FAKE_SEED_NUM,NODE_NUM, EDGE_NUM )

In [146]:
statistic_draw_graph(intial_train_graph_bench)

normal nodes number: 474
true nodes number: 0
fake nodes number: 26
the edge of graph: 1500


In [150]:
SEED_SIZE = int((FAKE_DIFF_ITER/(FAKE_DIFF_ITER+1))*len(active_fake_set_bench))
print(SEED_SIZE)

10


In [151]:
env_dqn = Env(intial_train_graph_bench, SEED_SIZE,active_fake_set_bench)

In [152]:
agent_dqn = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)

In [153]:
state_history_dqn, rewards_list_dqn = train_dqn(agent_dqn, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "dqn_bench.pth")

Episode 1/200, Total Reward: 25.296520152887528
Episode 2/200, Total Reward: 23.885668891946196
Episode 3/200, Total Reward: 17.885501733713827
Episode 4/200, Total Reward: 29.373353172983666


/var/folders/qd/bz9f8bp52sv9260n6zps4vbr0000gn/T/ipykernel_1713/545150970.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = torch.tensor(state, dtype=torch.float32).to(device)


Episode 5/200, Total Reward: 31.038097663786374
Episode 6/200, Total Reward: 31.953835445252523
Episode 7/200, Total Reward: 23.27227472472665
Episode 8/200, Total Reward: 30.609115009341455
Episode 9/200, Total Reward: 26.028641560458656
Episode 10/200, Total Reward: 33.21072660730408
Episode 11/200, Total Reward: 34.42503493403413
Episode 12/200, Total Reward: 31.91712791933004
Episode 13/200, Total Reward: 20.154257154215685
Episode 14/200, Total Reward: 30.334378062251304
Episode 15/200, Total Reward: 33.22760712823543
Episode 16/200, Total Reward: 29.10530369746765
Episode 17/200, Total Reward: 32.860362166208155
Episode 18/200, Total Reward: 32.75262739241275
Episode 19/200, Total Reward: 31.325943416379424
Episode 20/200, Total Reward: 30.402657916283736
Episode 21/200, Total Reward: 28.052880015818655
Episode 22/200, Total Reward: 25.752570330850734
Episode 23/200, Total Reward: 33.24457885952233
Episode 24/200, Total Reward: 29.610754878606482
Episode 25/200, Total Reward: 31.

Episode 172/200, Total Reward: 33.16199149584536
Episode 173/200, Total Reward: 25.78703310907659
Episode 174/200, Total Reward: 28.954048818709055
Episode 175/200, Total Reward: 24.261153196981574
Episode 176/200, Total Reward: 24.943103314892056
Episode 177/200, Total Reward: 31.859971838910024
Episode 178/200, Total Reward: 29.73241824588016
Episode 179/200, Total Reward: 18.042170243541165
Episode 180/200, Total Reward: 26.100278481461046
Episode 181/200, Total Reward: 29.035495633950987
Episode 182/200, Total Reward: 32.34004846524561
Episode 183/200, Total Reward: 28.795660767769853
Episode 184/200, Total Reward: 35.33450372370088
Episode 185/200, Total Reward: 32.981847964396
Episode 186/200, Total Reward: 30.397084429442142
Episode 187/200, Total Reward: 29.67299223291767
Episode 188/200, Total Reward: 31.57754170693525
Episode 189/200, Total Reward: 23.27784818411767
Episode 190/200, Total Reward: 31.196161279738195
Episode 191/200, Total Reward: 33.83518902610372
Episode 192/

In [64]:
#benchmark

In [154]:
agent_bench1 = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)
state_history_bench1, rewards_list_bench1 = train_dqn(agent_bench1, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "bench1.pth",brenchmark_action = 1)

Episode 1/200, Total Reward: 33.264029674919854
Episode 2/200, Total Reward: 24.77857379190233
Episode 3/200, Total Reward: 30.90999879614391
Episode 4/200, Total Reward: 27.56672955808226
Episode 5/200, Total Reward: 22.071397634680636
Episode 6/200, Total Reward: 26.303940836470776
Episode 7/200, Total Reward: 29.700066459026228
Episode 8/200, Total Reward: 26.61492726871452
Episode 9/200, Total Reward: 28.674885406577097
Episode 10/200, Total Reward: 19.96503381415827
Episode 11/200, Total Reward: 31.821645449667958
Episode 12/200, Total Reward: 32.28370181191792
Episode 13/200, Total Reward: 33.00360248702413
Episode 14/200, Total Reward: 32.987790276415645
Episode 15/200, Total Reward: 33.18924449101056
Episode 16/200, Total Reward: 26.37497031821486
Episode 17/200, Total Reward: 25.338174840668866
Episode 18/200, Total Reward: 28.282903444678965
Episode 19/200, Total Reward: 29.654685537411076
Episode 20/200, Total Reward: 31.757415357909355
Episode 21/200, Total Reward: 24.42445

Episode 169/200, Total Reward: 25.556952161057573
Episode 170/200, Total Reward: 25.07711022762799
Episode 171/200, Total Reward: 31.25920533536253
Episode 172/200, Total Reward: 27.52316551086556
Episode 173/200, Total Reward: 28.137488723253526
Episode 174/200, Total Reward: 21.99680128206125
Episode 175/200, Total Reward: 22.54050852612382
Episode 176/200, Total Reward: 27.123542786338845
Episode 177/200, Total Reward: 22.35520976979449
Episode 178/200, Total Reward: 28.425404771699938
Episode 179/200, Total Reward: 30.48217800572933
Episode 180/200, Total Reward: 26.613077025692828
Episode 181/200, Total Reward: 31.190068427855078
Episode 182/200, Total Reward: 27.385216145038257
Episode 183/200, Total Reward: 32.877932210866895
Episode 184/200, Total Reward: 30.82424146293084
Episode 185/200, Total Reward: 30.635245674886146
Episode 186/200, Total Reward: 32.98692282623051
Episode 187/200, Total Reward: 26.637377892726434
Episode 188/200, Total Reward: 21.19530888988629
Episode 18

In [155]:
agent_bench2 = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)
state_history_bench2, rewards_list_bench2 = train_dqn(agent_bench2, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "bench2.pth",brenchmark_action = 2)

Episode 1/200, Total Reward: 26.721969786367683
Episode 2/200, Total Reward: 28.49125295589617
Episode 3/200, Total Reward: 22.083073516936924
Episode 4/200, Total Reward: 28.963153546375885
Episode 5/200, Total Reward: 28.809422564988672
Episode 6/200, Total Reward: 29.120725034867764
Episode 7/200, Total Reward: 32.16651491236425
Episode 8/200, Total Reward: 24.341877713400148
Episode 9/200, Total Reward: 22.479955620652778
Episode 10/200, Total Reward: 32.94510636292413
Episode 11/200, Total Reward: 26.719174961319904
Episode 12/200, Total Reward: 32.16060052951385
Episode 13/200, Total Reward: 30.133896837929868
Episode 14/200, Total Reward: 29.95705310671065
Episode 15/200, Total Reward: 32.55235098941451
Episode 16/200, Total Reward: 32.40214375721577
Episode 17/200, Total Reward: 27.38640885750213
Episode 18/200, Total Reward: 32.451515094882936
Episode 19/200, Total Reward: 30.57559135705656
Episode 20/200, Total Reward: 29.926886795986743
Episode 21/200, Total Reward: 31.61567

Episode 169/200, Total Reward: 28.61914800732406
Episode 170/200, Total Reward: 30.997157360986247
Episode 171/200, Total Reward: 27.57910640749631
Episode 172/200, Total Reward: 25.150708297001866
Episode 173/200, Total Reward: 26.766328093597984
Episode 174/200, Total Reward: 27.91510749857956
Episode 175/200, Total Reward: 26.507841427294366
Episode 176/200, Total Reward: 30.0200512816794
Episode 177/200, Total Reward: 30.949200899718804
Episode 178/200, Total Reward: 29.08397256952869
Episode 179/200, Total Reward: 28.191984090363146
Episode 180/200, Total Reward: 27.53585937679997
Episode 181/200, Total Reward: 35.537858318644126
Episode 182/200, Total Reward: 31.282479576543867
Episode 183/200, Total Reward: 27.478889857122784
Episode 184/200, Total Reward: 31.188343464670215
Episode 185/200, Total Reward: 27.53927542387002
Episode 186/200, Total Reward: 23.1681254890188
Episode 187/200, Total Reward: 32.256434208107464
Episode 188/200, Total Reward: 28.94092377083882
Episode 189

In [156]:
agent_bench3 = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)
state_history_bench3, rewards_list_bench3 = train_dqn(agent_bench3, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "bench3.pth",brenchmark_action = 3)

Episode 1/200, Total Reward: 27.85364371288117
Episode 2/200, Total Reward: 35.542641235251565
Episode 3/200, Total Reward: 22.7195396132626
Episode 4/200, Total Reward: 30.2547506986412
Episode 5/200, Total Reward: 31.246649805741587
Episode 6/200, Total Reward: 34.100866236751514
Episode 7/200, Total Reward: 28.711593670854658
Episode 8/200, Total Reward: 25.01698475230285
Episode 9/200, Total Reward: 29.926907505741188
Episode 10/200, Total Reward: 28.62562522363876
Episode 11/200, Total Reward: 29.398104955720907
Episode 12/200, Total Reward: 30.742214517367536
Episode 13/200, Total Reward: 31.353363913238788
Episode 14/200, Total Reward: 33.21029403198849
Episode 15/200, Total Reward: 31.40580064514256
Episode 16/200, Total Reward: 28.23477671071608
Episode 17/200, Total Reward: 31.745549311611974
Episode 18/200, Total Reward: 27.883904120222006
Episode 19/200, Total Reward: 27.19951794881924
Episode 20/200, Total Reward: 32.63720451730601
Episode 21/200, Total Reward: 31.40206844

Episode 169/200, Total Reward: 31.702997552659028
Episode 170/200, Total Reward: 31.87817517835823
Episode 171/200, Total Reward: 22.426782792669993
Episode 172/200, Total Reward: 32.41999501464213
Episode 173/200, Total Reward: 30.336514607734827
Episode 174/200, Total Reward: 28.43557238512452
Episode 175/200, Total Reward: 29.54408837828524
Episode 176/200, Total Reward: 28.324488512625496
Episode 177/200, Total Reward: 30.942427924362235
Episode 178/200, Total Reward: 34.83308251183864
Episode 179/200, Total Reward: 30.931265299308315
Episode 180/200, Total Reward: 27.905710337048657
Episode 181/200, Total Reward: 30.495602881817288
Episode 182/200, Total Reward: 32.759661308853055
Episode 183/200, Total Reward: 33.91123385744746
Episode 184/200, Total Reward: 22.18650317531885
Episode 185/200, Total Reward: 35.28836227366582
Episode 186/200, Total Reward: 32.98852699458449
Episode 187/200, Total Reward: 28.196266860215772
Episode 188/200, Total Reward: 25.04368956344777
Episode 18

In [157]:
agent_bench4 = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)
state_history_bench4, rewards_list_bench4 = train_dqn(agent_bench4, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "bench4.pth",brenchmark_action = 4)

Episode 1/200, Total Reward: 32.079737175591845
Episode 2/200, Total Reward: 30.306366759220406
Episode 3/200, Total Reward: 27.91211712825688
Episode 4/200, Total Reward: 27.931258941129965
Episode 5/200, Total Reward: 27.510831917326843
Episode 6/200, Total Reward: 29.55307606439297
Episode 7/200, Total Reward: 30.15783318017988
Episode 8/200, Total Reward: 34.363108328668105
Episode 9/200, Total Reward: 32.59777077587986
Episode 10/200, Total Reward: 24.577823575896048
Episode 11/200, Total Reward: 29.447761641317953
Episode 12/200, Total Reward: 31.534197368187705
Episode 13/200, Total Reward: 31.191676738318044
Episode 14/200, Total Reward: 34.77918959410782
Episode 15/200, Total Reward: 30.003915446508373
Episode 16/200, Total Reward: 30.538523231995537
Episode 17/200, Total Reward: 31.591649017100952
Episode 18/200, Total Reward: 32.06383687167952
Episode 19/200, Total Reward: 31.341316861054793
Episode 20/200, Total Reward: 29.924504236560388
Episode 21/200, Total Reward: 32.58

Episode 169/200, Total Reward: 32.77966751073865
Episode 170/200, Total Reward: 20.455238052371627
Episode 171/200, Total Reward: 29.790775762289808
Episode 172/200, Total Reward: 34.867565174339255
Episode 173/200, Total Reward: 32.00371802940005
Episode 174/200, Total Reward: 31.47308944245852
Episode 175/200, Total Reward: 30.47648342036461
Episode 176/200, Total Reward: 30.782997457566516
Episode 177/200, Total Reward: 32.82455388593986
Episode 178/200, Total Reward: 25.124848875257552
Episode 179/200, Total Reward: 28.929153307017486
Episode 180/200, Total Reward: 32.52825074236026
Episode 181/200, Total Reward: 28.37869737610055
Episode 182/200, Total Reward: 31.00660070108902
Episode 183/200, Total Reward: 33.79227797599676
Episode 184/200, Total Reward: 22.294524818005467
Episode 185/200, Total Reward: 32.019903060386554
Episode 186/200, Total Reward: 29.686920006794676
Episode 187/200, Total Reward: 27.950581881394974
Episode 188/200, Total Reward: 28.3014865252696
Episode 189

In [158]:
agent_bench5 = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)
state_history_bench5, rewards_list_bench5 = train_dqn(agent_bench5, env_dqn, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name = "bench5.pth",brenchmark_action = 5)

Episode 1/200, Total Reward: 27.159880399591565
Episode 2/200, Total Reward: 28.87385051601075
Episode 3/200, Total Reward: 30.938000693234134
Episode 4/200, Total Reward: 30.845696715044333
Episode 5/200, Total Reward: 30.318991380631243
Episode 6/200, Total Reward: 26.55458959890495
Episode 7/200, Total Reward: 31.14872821749398
Episode 8/200, Total Reward: 26.964902215590584
Episode 9/200, Total Reward: 33.54574422269999
Episode 10/200, Total Reward: 27.705236158341965
Episode 11/200, Total Reward: 32.33281842927475
Episode 12/200, Total Reward: 32.88933694977747
Episode 13/200, Total Reward: 30.305352064816226
Episode 14/200, Total Reward: 30.656243943322348
Episode 15/200, Total Reward: 31.97978231177624
Episode 16/200, Total Reward: 28.31734524151837
Episode 17/200, Total Reward: 29.732058448697572
Episode 18/200, Total Reward: 31.05865274768216
Episode 19/200, Total Reward: 30.088560216447203
Episode 20/200, Total Reward: 27.614172761028037
Episode 21/200, Total Reward: 32.63941

Episode 169/200, Total Reward: 29.019319840343723
Episode 170/200, Total Reward: 27.924265248590327
Episode 171/200, Total Reward: 19.575725312968242
Episode 172/200, Total Reward: 30.52112511573922
Episode 173/200, Total Reward: 32.20560149824374
Episode 174/200, Total Reward: 34.26208817495478
Episode 175/200, Total Reward: 22.966417918167366
Episode 176/200, Total Reward: 31.452246039702377
Episode 177/200, Total Reward: 29.700444569970305
Episode 178/200, Total Reward: 30.14702816196124
Episode 179/200, Total Reward: 30.8266572773562
Episode 180/200, Total Reward: 31.216114416807407
Episode 181/200, Total Reward: 28.915220726951862
Episode 182/200, Total Reward: 26.583537827588508
Episode 183/200, Total Reward: 26.96153598527792
Episode 184/200, Total Reward: 19.179712474590637
Episode 185/200, Total Reward: 26.277460296311247
Episode 186/200, Total Reward: 24.16663347977751
Episode 187/200, Total Reward: 33.14652936288954
Episode 188/200, Total Reward: 31.5426721897336
Episode 189